Simple OSEM reconstruction demo for real data: 
This notebook use data acquired at NPL and it can be downloaded at https://osf.io/pcfb4/. The experiment consists in a Nema phantom with hot spherical inserts (filled with Y90), a cold background, and a lung equivalent cilindrical insert. More information about the experiment can be found in the following manuscripts:
Deidda et al, EJNMMI, 2022. https://ejnmmiphys.springeropen.com/articles/10.1186/s40658-022-00452-4

This demo is a jupyter notebook, i.e. intended to be run step by step.

Authors: Daniel Deidda, Sam Porter, Kris Thielemans

First version: 13th of May 2022 

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2022 National Physical Laboratory.  
Copyright 2022 University College London.  

This is software developed for the Collaborative Computational
Project in Synergistic Reconstruction for Biomedical Imaging
(http://www.ccpsynerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

In this exercise you are going to apply what you learned in the previous notebooks about SPECT reconstruction to reconstruct real data. 


In [ ]:
# Initial imports etc
import os
import glob
import numpy as np
import subprocess as sp

# import engine module
# Setup the working directory for the notebook
import notebook_setup
from sirf_exercises import cd_to_working_dir
from sirf.Utilities import examples_data_path
from sirf.STIR import show_2D_array
from sirf.STIR import MessageRedirector

import sirf.STIR as spect
cd_to_working_dir('measured_SPECT')
msg_red = MessageRedirector('info.txt', 'warnings.txt', 'errors.txt')


In [ ]:
#move data to working_folder and show where we are
!pwd
!cp -r /mnt/materials/SIRF/PSMRTBP2022/Y90-NEMA-NPL/SPECT/* .
!ls .

The following is not needed if the data is already downloaded

In [ ]:
# Install OSF client to download the NPL data from https://osf.io/pcfb4/ this is a NEMA phantom with spherical inserts filled with Y90
#bash_install = "pip install osfclient"
#process = sp.Popen(bash_install.split(), stdout=sp.PIPE)
#output, error = process.communicate()


In [ ]:
#  Actuall download
#bash_clone = "osf -p pcfb4 clone"
#process = sp.Popen(bash_clone.split(), stdout=sp.PIPE)
#output, error = process.communicate()

In [ ]:
# set data template
templ_sino = spect.AcquisitionData('peak_1_projdata__f1g1d0b0.hs')

# Exercise 1: Simple Reconstruction:
Once you read the measured sinogram implement the system model end set up the reconstruction model and run a reconstruction

In [ ]:
# plot measured sinogram
measured_data_as_array = templ_sino.as_array()
middle_slice=measured_data_as_array.shape[2]//2
print(middle_slice)
show_2D_array('Measured data', measured_data_as_array[0, middle_slice,:,:])
measured_data_as_array.min()

In [ ]:
# create image with suitable sizes
image = templ_sino.create_uniform_image()
print(image.voxel_sizes())
print(image.dimensions()) 

In [ ]:
# show the attenuation image
uMap = spect.ImageData('umap_zoomed.hv')
z = uMap.dimensions()[0]//2 -10
uMap_array = uMap.as_array()
uMap.dimensions()
show_2D_array('Attenuation image', uMap_array[z,:,:])

In [ ]:
# select acquisition model that implements the geometric
# forward projection by a ray tracing matrix multiplication
acq_model_matrix = spect.SPECTUBMatrix();
#acq_model_matrix.set_keep_all_views_in_cache(True)
acq_model_matrix.set_attenuation_image(uMap) # add attenuation
acq_model = spect.AcquisitionModelUsingMatrix(acq_model_matrix)

In [ ]:
# create objective function
obj_fun = spect.make_Poisson_loglikelihood(templ_sino)
obj_fun.set_acquisition_model(acq_model)

In [ ]:
# create OSEM reconstructor object
num_subsets = 12 # number of subsets for OSEM reconstruction
num_subiters = 12 #number of subiterations (i.e one full iterations)
OSEM_reconstructor = spect.OSMAPOSLReconstructor()
OSEM_reconstructor.set_objective_function(obj_fun)
OSEM_reconstructor.set_num_subsets(num_subsets)
OSEM_reconstructor.set_num_subiterations(num_subiters)

In [ ]:
# create initialisation image and set up reconstructor
# the following lines are needed to make the initialisation image compatible with the uMap
offset=(image.dimensions()[2]-2)*image.voxel_sizes()[2]/2 - (image.dimensions()[2]-1)*image.voxel_sizes()[2]/2
init_image = image.zoom_image(zooms=(0.5, 1.0, 1.0), offsets_in_mm=(0,offset,offset), size=(128,128,128))
init_image.write("init.hv")
OSEM_reconstructor.set_up(init_image.get_uniform_copy(1))


In [ ]:
# Reconstruct and show reconstructed image
OSEM_reconstructor.reconstruct(init_image.get_uniform_copy(1))
out_image = OSEM_reconstructor.get_current_estimate()
out_image_array = out_image.as_array()
show_2D_array('Reconstructed image', out_image_array[z,:,:])

# Exercise 2: PSF Reconstruction:
2a) now change the acquisition model to include resolution modelling (use help(SPECTUBMatrix)) to find the right function. The resolution model in STIR is based on Gaussian function and assumes linear dependency of the sigma from the distance of the detector Sigma(d) = slope*d +Sigma0.

The measured slope and sigma0 for these data is respectevely 0.027 and 0.417 (cm)

2b) study the difference between OSEM and OSEM-PSF

2c) study also the difference between using a fully 3D PSF or a 2D PSF

In [ ]:
#2a
acq_model_matrix.set_resolution_model(4.17,0.27,False)
obj_fun.set_acquisition_model(acq_model)
OSEM_reconstructor.set_objective_function(obj_fun)
OSEM_reconstructor.set_up(init_image.get_uniform_copy(1))
OSEM_reconstructor.reconstruct(init_image.get_uniform_copy(1))
out_image_psf = OSEM_reconstructor.get_current_estimate()
out_image_psf_array = out_image_psf.as_array()
#2b
diff_array=out_image_array-out_image_psf_array
show_2D_array('Reconstructed image with PSF', out_image_psf_array[z,:,:])
show_2D_array('Reconstructed image', out_image_array[z,:,:])
show_2D_array('OSEM - OSEM-PSF', diff_array[z,:,:])

In [ ]:
#2c
acq_model_matrix.set_resolution_model(4.17,0.27,True)
obj_fun.set_acquisition_model(acq_model)
OSEM_reconstructor.set_objective_function(obj_fun)
OSEM_reconstructor.set_up(init_image.get_uniform_copy(1))
OSEM_reconstructor.reconstruct(init_image.get_uniform_copy(1))
out_image_psf3d = OSEM_reconstructor.get_current_estimate()
out_image_psf3d_array = out_image_psf3d.as_array()
diff2_array=out_image_psf3d_array-out_image_psf_array
show_2D_array('Reconstructed image with PSF 3D', out_image_psf3d_array[z,:,:])
show_2D_array('Reconstructed image', out_image_psf_array[z,:,:])
show_2D_array('OSEM-PSF3D - OSEM-PSF2D', diff2_array[z,:,:])